## Machine Learning 
- scikit pipeline


In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('../data/ABtwoCampaignEngView.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912712 entries, 0 to 912711
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   912712 non-null  int64  
 1   auction_id   912712 non-null  object 
 2   experiment   912712 non-null  object 
 3   date         912712 non-null  object 
 4   hour         912712 non-null  int64  
 5   device_make  912712 non-null  object 
 6   platform_os  912712 non-null  int64  
 7   browser      912712 non-null  int64  
 8   yes          912712 non-null  float64
 9   no           912712 non-null  float64
dtypes: float64(2), int64(4), object(4)
memory usage: 69.6+ MB


In [22]:
df['yes'] = df['yes'].astype('int')

In [23]:
from sklearn.model_selection import train_test_split
X = df.drop(['yes'], axis=1)
y = df[['yes']]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123)


In [24]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [25]:
class columnDropperTransformer():
    def __init__(self, columns):
        self.columns = columns

    def transform(self, X, y=None):
        return X.drop([self.columns], axis=1)

    def fit(self, X, y=None):
        return self


# class CategoricalTransformer():
#     def __init__(self, columns):
#         self.columns = columns

#     def transform(self, X, y=None):
#         encoder = LabelEncoder()
#         for col in self.columns:
#             X[col] = encoder.fit_transform(X[col])
        
#         return X

#     def fit(self, X, y=None):
#         return self


In [26]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')), ('encoder', LabelEncoder())
])


In [27]:

categorical_features = ['device_make', 'platform_os', 'browser']
preprocessor = ColumnTransformer(
    transformers=[
         ('categorical', categorical_transformer, categorical_features),
         ("columnDropper", columnDropperTransformer(
             ['hour', 'date', 'Unnamed: 0', 'auction_id','no']))
         
    ])


In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), ('classifier', LogisticRegression())
])

In [29]:
rf_model = pipeline.fit(X_train, y_train)
print(rf_model)

from sklearn.metrics import accuracy_score
predictions = rf_model.predict(X_test)
print (accuracy_score(y_test, predictions))

ValueError: not enough values to unpack (expected 3, got 2)

## Use the model

In [ ]:
import joblib
joblib.dump(rf_model, './rf_model.pkl')

In [ ]:
# In other notebooks
rf_model = joblib.load('PATH/TO/rf_model.pkl')
new_prediction = rf_model.predict(new_data)
